In [2]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import keras

input_shape = (32, 32, 3)
img_width = 32
img_height = 32
num_classes = 10
nb_train_samples = 10000
nb_validation_samples = 2000
batch_size = 16
epochs = 1

train_data_dir = './credit_card/train'
validation_data_dir = './credit_card/test'

# Creating our data generator for our test data
validation_datagen = ImageDataGenerator(
    # used to rescale the pixel values from [0, 255] to [0, 1] interval
    rescale = 1./255)

# Creating our data generator for our training data
train_datagen = ImageDataGenerator(
      rescale = 1./255,              # normalize pixel values to [0,1]
      rotation_range = 10,           # randomly applies rotations
      width_shift_range = 0.25,       # randomly applies width shifting
      height_shift_range = 0.25,      # randomly applies height shifting
      shear_range=0.5,
      zoom_range=0.5,
      horizontal_flip = False,        # randonly flips the image
      fill_mode = 'nearest')         # uses the fill mode nearest to fill gaps created by the above

# Specify criteria about our training data, such as the directory, image size, batch size and type 
# automagically retrieve images and their classes for train and validation sets
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = 'categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = False)    

Found 20000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


In [22]:
# create model
model = Sequential()

# 2 sets of CRP (Convolution, RELU, Pooling)
model.add(Conv2D(20, (5, 5),
                 padding = "same", 
                 input_shape = input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

model.add(Conv2D(50, (5, 5),
                 padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

# Fully connected layers (w/ RELU)
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

# Softmax (for classification)
model.add(Dense(num_classes))
model.add(Activation("softmax"))
           
model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])
    
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 20)        1520      
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 20)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 20)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 50)        25050     
_________________________________________________________________
activation_6 (Activation)    (None, 16, 16, 50)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 50)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3200)              0         
__________

In [23]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("/home/deeplearningcv/DeepLearningCV/Trained Models/creditcard.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 20000
nb_validation_samples = 4000
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save("/home/deeplearningcv/DeepLearningCV/Trained Models/creditcard.h5")

Epoch 1/5
1250/1250 [==============================] - 105s 84ms/step - loss: 0.4301 - acc: 0.8571 - val_loss: 0.0022 - val_acc: 1.0000

Epoch 00001: val_loss improved from inf to 0.00220, saving model to /home/deeplearningcv/DeepLearningCV/Trained Models/creditcard.h5
Epoch 2/5
1250/1250 [==============================] - 116s 93ms/step - loss: 0.1545 - acc: 0.9494 - val_loss: 0.0014 - val_acc: 0.9998

Epoch 00002: val_loss improved from 0.00220 to 0.00135, saving model to /home/deeplearningcv/DeepLearningCV/Trained Models/creditcard.h5
Epoch 3/5
1250/1250 [==============================] - 170s 136ms/step - loss: 0.1200 - acc: 0.9596 - val_loss: 0.0015 - val_acc: 0.9995

Epoch 00003: val_loss did not improve from 0.00135
Epoch 4/5
1250/1250 [==============================] - 158s 126ms/step - loss: 0.1046 - acc: 0.9672 - val_loss: 0.0093 - val_acc: 0.9990

Epoch 00004: val_loss did not improve from 0.00135
Epoch 5/5
1250/1250 [==============================] - 113s 90ms/step - loss: 

In [4]:
from keras.models import load_model

classifier = load_model('/home/deeplearningcv/DeepLearningCV/Trained Models/creditcard.h5')

In [27]:
import cv2
from keras.preprocessing import image

def pre_process(image, inv = False):
    try:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except:
        gray_image = image
        pass

    kernel = np.ones((3,3), np.uint8)
    
    if inv == False:
        _, th2 = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    else:
        _, th2 = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    #dilation = cv2.dilate(th2, kernel, iterations = 1)
    resized = cv2.resize(th2, (32,32), interpolation = cv2.INTER_AREA)
    return resized

image = cv2.imread('credit_card.jpg')
resized = cv2.resize(image, (640,403), interpolation = cv2.INTER_AREA)

ROI = ([(66, 220), (92, 262)], [(92, 12), (295, 25)])
region = ROI[0]

top_left_y = region[0][1]
bottom_right_y = region[1][1]
top_left_x = region[0][0]
bottom_right_x = region[1][0]

for i in range(0,16):
    if i > 0 and i%4  == 0:
        jump = 30
    else:
        jump = 0
    
    if i > 0:
        top_left_x = top_left_x + 26 + jump
        bottom_right_x = bottom_right_x + 26 + jump

    roi = resized[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
    roi_otsu = pre_process(roi)
    cv2.imshow("roi", roi)
    cv2.imshow("roi_otsu", roi_otsu)
    roi_otsu = cv2.cvtColor(roi_otsu, cv2.COLOR_GRAY2RGB)
    x = keras.preprocessing.image.img_to_array(roi_otsu)
    x = x * 1./255
    x = np.expand_dims(x, axis=0)
    image = np.vstack([x])
    label = classifier.predict_classes(image, batch_size = 10)
    print(label)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

[5]
[4]
[7]
[6]
[7]
[6]
[7]
[8]
[9]
[8]
[7]
[6]
[5]
[4]
[3]
[2]


In [29]:
cv2.destroyAllWindows()

In [13]:
def pre_process(image, inv = False):
    try:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except:
        gray_image = image
        pass

    kernel = np.ones((3,3), np.uint8)
    
    if inv == False:
        _, th2 = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    else:
        _, th2 = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    #dilation = cv2.dilate(th2, kernel, iterations = 1)
    resized = cv2.resize(th2, (32,32), interpolation = cv2.INTER_AREA)
    return resized

def x_cord_contour(contour):
    # This function take a contour from findContours
    # it then outputs the x centroid coordinates
    M = cv2.moments(contour)
    return (int(M['m10']/M['m00']))

In [39]:
img = cv2.imread('card_numbers.jpg')
orig_img = cv2.imread('credit_card_color.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imshow("image", img)
cv2.waitKey(0)

# Blur image then find edges using Canny 
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#cv2.imshow("blurred", blurred)
#cv2.waitKey(0)

edged = cv2.Canny(blurred, 30, 150)
#cv2.imshow("edged", edged)
#cv2.waitKey(0)

# Find Contours
_, contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#Sort out contours left to right by using their x cordinates
contours = sorted(contours, key = x_cord_contour, reverse = False)

# Create empty array to store entire number
full_number = []

# loop over the contours
for c in contours:
    # compute the bounding box for the rectangle
    (x, y, w, h) = cv2.boundingRect(c)    

    if w >= 5 and h >= 25:
        roi = blurred[y:y + h, x:x + w]
        #ret, roi = cv2.threshold(roi, 20, 255,cv2.THRESH_BINARY_INV)
        cv2.imshow("ROI1", roi)
        roi_otsu = pre_process(roi, True)
        cv2.imshow("ROI2", roi_otsu)
        roi_otsu = cv2.cvtColor(roi_otsu, cv2.COLOR_GRAY2RGB)
        roi_otsu = keras.preprocessing.image.img_to_array(roi_otsu)
        roi_otsu = roi_otsu * 1./255
        roi_otsu = np.expand_dims(roi_otsu, axis=0)
        image = np.vstack([roi_otsu])
        label = str(classifier.predict_classes(image, batch_size = 10))
        print(label)
        cv2.rectangle(orig_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(orig_img, label, (x , y + 155), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 1)
        cv2.imshow("image", orig_img)
        cv2.waitKey(0) 
        
cv2.destroyAllWindows()

[5]
[3]
[5]
[5]
[2]
[2]
[0]
[3]
[2]
[3]
[9]
[0]
[0]
[7]
[5]
[7]


In [35]:
(x, y, w, h)

(549, 7, 25, 38)